In [9]:
import cv2
import numpy as np

# Open video capture
cap = cv2.VideoCapture('machine_output.avi')

# Define optical flow parameters
feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
lk_params = dict(winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Initialize previous frame and points
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
prev_points = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)

# Define threshold value
threshold = 0.5

while True:
    # Capture current frame and convert to grayscale
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow
    curr_points, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None, **lk_params)

    # Calculate magnitude of optical flow
    mag = cv2.magnitude(curr_points - prev_points, np.zeros_like(curr_points))

    # Check if magnitude exceeds threshold
    if np.mean(mag) > threshold:
        print("Warning: Machine is faulty!")
        break

    # Update previous frame and points
    prev_gray = gray.copy()
    prev_points = curr_points

cap.release()
cv2.destroyAllWindows()
